In [1]:
# Bibliotecas
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import xlwings as xw
import pandas as pd

In [2]:
# Dados
wb = xw.Book('dados_1.xlsx')
sheet = wb.sheets['Plan1']
# Conjuntos
Clientes = sheet.range("C2:G2").value
n = len(Clientes)  # Número de clientes (5)
Veiculos = sheet.range("A12:A14").value
Nos = [i for i in range(n + 2)]
# Parâmetros
custo_transporte = sheet.range("A2:H9").options(pd.DataFrame).value
custo_transporte = custo_transporte.stack().to_dict()
demanda = sheet.range("D12:E18").options(dict).value
capacidade = sheet.range("A12:B14").options(dict).value

In [3]:
# Modelo
model = ConcreteModel()

# Conjuntos
model.C = Set(initialize=Clientes)
model.K = Set(initialize=Veiculos)
model.N = Set(initialize=Nos)
C, K, N = model.C, model.K, model.N

# Parâmetros
model.Q = Param(K, initialize=capacidade, within=NonNegativeReals)
model.d = Param(N, initialize=demanda, within=NonNegativeReals)
model.c = Param(N * N, initialize=custo_transporte, within=NonNegativeReals)
Q, d, c = model.Q, model.d, model.c

# Variáveis de decisão
model.x = Var(N * N * K, within=Binary)
x = model.x

# Função objetivo
model.obj = Objective(expr=sum(sum(sum(c[i, j] * x[i, j, k] for j in N) for i in N) for k in K))

# Restrições

# 1) Cada cliente visitado uma vez
model.r_visita = ConstraintList()
for i in C:
    model.r_visita.add(expr=sum(sum(x[i, j, k] for j in N if (j > 0 and j != i)) for k in K) == 1)
    
# 2) O mesmo veículo visita antecessor/sucessor
model.r_fluxo = ConstraintList()
for h in C:
    for k in K:
        model.r_fluxo.add(expr=sum(x[i, h, k] for i in N if (i < (n + 1) and i != h)) == 
                                   sum(x[h, j, k] for j in N if (j > 0 and j != h)))

# 3) Capacidade máxima do veículo
model.r_capacidade = ConstraintList()
for k in K:
    model.r_capacidade.add(expr=sum(sum(d[i] * x[i, j, k] for i in C) for j in N if (j != i and j > 0)) <= Q[k])

# 4) Cada veículo deve sair do depósito
model.r_saida = ConstraintList()
for k in K:
    model.r_saida.add(expr=sum(x[0, j, k] for j in N if j > 0) == 1)

# 5) Cada veículo deve retornar ao depósito
model.r_retorno = ConstraintList()
for k in K:
    model.r_retorno.add(expr=sum(x[i, n + 1, k] for i in N if i < (n + 1)) == 1)

In [4]:
# # Sub rotas
# sub1 = [0, 6]
# sub2 = [1, 2]
# sub3 = [3, 4, 5]
# sub4 = [3, 4]
# sub5 = [4, 5]
# sub6 = [1, 3, 2]
# sub7 = [3, 5]
# sub8 = [1, 2, 4]
# sub9 = [3, 4, 2, 1]
# sub10 = [1, 3]

# sub_rotas = [sub1, sub2, sub3, sub4, sub5, sub6, sub7, sub8, sub9, sub10]

# # Restrições para as subrotas dos nós
# model.constraints = ConstraintList()
# for sub in sub_rotas:
#     for k in K:
#         model.constraints.add(expr = sum(sum(x[i,j,k] for i in sub) for j in sub) <= len(sub) - 1)

In [5]:
# Formulação de MTZ
model.u = Var(N * K, within=NonNegativeReals)
u = model.u

model.r_u = ConstraintList()
for i in C:
    for k in K:
        model.r_u.add(expr=u[i,k] <= Q[k])
for i in N:
    if N != n + 1:
        for j in C:
            if i != j:
                for k in K:
                    model.r_u.add(expr=u[j,k] >= u[i,k] + d[j]*x[i,j,k] + Q[k]*(x[i,j,k] - 1))

In [6]:
# Resolução
solver = SolverFactory('cplex')
resultado = solver.solve(model, tee = False)

In [7]:
if (resultado.solver.status == SolverStatus.ok) and (resultado.solver.termination_condition != TerminationCondition.infeasible):
    print(f'Custo mínimo: R${value(model.obj)}')
    print()
    for k in K:
        for i in N:
            if i < n + 1:
                for j in N:
                    if i != j:
                        if j > 0:
                            if value(model.x[i,j,k]) == 1:
                                print(f'O {k} vai de {i} para {j}')
else:
     print(resultado.solver.termination_condition)

Custo mínimo: R$67.0

O Caminhao 1 vai de 0 para 1
O Caminhao 1 vai de 1 para 2
O Caminhao 1 vai de 2 para 6
O Caminhao 2 vai de 0 para 5
O Caminhao 2 vai de 3 para 4
O Caminhao 2 vai de 4 para 6
O Caminhao 2 vai de 5 para 3
O Caminhao 3 vai de 0 para 6
